In [1]:
import cv2
import yaml
import tensorflow as tf

import numpy as np
import yaml

In [2]:
tf_model = "SSD1/lite-model_ssd_mobilenet_v1_1_metadata_2.tflite"

In [3]:
interpreter = tf.lite.Interpreter(model_path=tf_model)
interpreter.allocate_tensors()
# Get the model inputs
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
input_details

[{'name': 'normalized_input_image_tensor',
  'index': 175,
  'shape': array([  1, 300, 300,   3], dtype=int32),
  'shape_signature': array([  1, 300, 300,   3], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.0078125, 128),
  'quantization_parameters': {'scales': array([0.0078125], dtype=float32),
   'zero_points': array([128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [5]:
for i in output_details:
    print(i)

{'name': 'TFLite_Detection_PostProcess', 'index': 167, 'shape': array([ 1, 10,  4], dtype=int32), 'shape_signature': array([ 1, 10,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'TFLite_Detection_PostProcess:1', 'index': 168, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'TFLite_Detection_PostProcess:2', 'index': 169, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), '

In [11]:

def preprocess(path):
    """
    Preprocesses the input image before performing inference.

    Returns:
        image_data: Preprocessed image data ready for inference.
    """
    input_width = 300
    input_height = 300
    # Read the input image using OpenCV
    img = cv2.imread(path)

    # Get the height and width of the input image
    img_height, img_width = img.shape[:2]

    # Convert the image color space from BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resize the image to match the input shape
    img = cv2.resize(img, (input_width, input_height))

    # Normalize the image data by dividing it by 255.0
    image_data = np.array(img) / 255.0

    # Transpose the image to have the channel dimension as the first dimension
    # image_data = np.transpose(image_data, (2, 0, 1))  # Channel first

    # Expand the dimensions of the image data to match the expected input shape
    image_data = np.expand_dims(image_data, axis=0).astype(np.uint8)

    # Return the preprocessed image data
    return image_data


In [12]:
img_data = preprocess("car_person.jpg")

In [13]:
interpreter.set_tensor(input_details[0]['index'], img_data )
interpreter.invoke()
box_data = interpreter.get_tensor(output_details[0]['index'])
class_data = interpreter.get_tensor(output_details[1]['index'])
score_data = interpreter.get_tensor(output_details[2]['index'])
N_data = interpreter.get_tensor(output_details[3]['index'])

In [14]:
box_data

array([[[ 2.8988817e-01,  1.5049994e-02,  9.8729146e-01,  7.9404360e-01],
        [ 3.3722758e-02,  1.1624396e-03,  9.9431860e-01,  9.9260890e-01],
        [ 3.8548976e-01,  1.3405573e-01,  9.9042028e-01,  9.9054027e-01],
        [ 6.1274660e-01,  2.6420742e-02,  9.8115492e-01,  9.8509765e-01],
        [ 6.5799445e-02, -7.0050359e-04,  9.6387386e-01,  5.8540654e-01],
        [ 4.2022616e-02, -1.0624081e-03,  9.9275041e-01,  2.5891066e-01],
        [-1.2793273e-02, -1.6335785e-02,  2.4310306e-01,  9.9643987e-01],
        [ 3.8660854e-02,  5.9224522e-01,  9.6959364e-01,  1.0291947e+00],
        [ 2.8355390e-02,  7.3278981e-01,  9.7908318e-01,  9.9690455e-01],
        [ 9.1650784e-03,  2.8540042e-01,  6.7427146e-01,  9.8280370e-01]]],
      dtype=float32)

In [15]:
class_data

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [21]:
sum(score_data[0])

0.99609375

In [16]:
N_data 

array([10.], dtype=float32)

In [ ]:
class TFinference:
    """YOLOv8 object detection model class for handling inference and visualization."""

    def __init__(self, tf_model, input_image, confidence_thres, iou_thres,cocolabel_path,positive_list):
        """
        Initializes an instance of the YOLOv8 class.

        Args:
            onnx_model: Path to the ONNX model.
            input_image: Path to the input image.
            confidence_thres: Confidence threshold for filtering detections.
            iou_thres: IoU (Intersection over Union) threshold for non-maximum suppression.
        """
        
        self.tf_model = tf_model
        self.input_image = input_image
        self.confidence_thres = confidence_thres
        self.iou_thres = iou_thres
        self.cocolabel_path = cocolabel_path

        # Load the class names from the COCO dataset
        self.classes = self.yaml_load()['names']
        
        # Generate a color palette for the classes
        self.color_palette = np.random.uniform(0, 255, size=(len(self.classes), 3))
        self.positive_list = positive_list

    def draw_detections(self, img, box, score, class_id):
        """
        Draws bounding boxes and labels on the input image based on the detected objects.

        Args:
            img: The input image to draw detections on.
            box: Detected bounding box.
            score: Corresponding detection score.
            class_id: Class ID for the detected object.

        Returns:
            None
        """

        # Extract the coordinates of the bounding box
        x1, y1, w, h = box

        # Retrieve the color for the class ID
        color = self.color_palette[class_id]

        # Draw the bounding box on the image
        cv2.rectangle(img, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

        # Create the label text with class name and score
        label = f'{self.classes[class_id]}: {score:.2f}'

        # Calculate the dimensions of the label text
        (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

        # Calculate the position of the label text
        label_x = x1
        label_y = y1 - 10 if y1 - 10 > label_height else y1 + 10

        # Draw a filled rectangle as the background for the label text
        cv2.rectangle(img, (label_x, label_y - label_height), (label_x + label_width, label_y + label_height), color,
                    cv2.FILLED)

        # Draw the label text on the image
        cv2.putText(img, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

    def yaml_load(self):
        with open(self.cocolabel_path, 'r') as file:
            cfg = yaml.safe_load(file)
        return cfg
    
    def preprocess(self):
        """
        Preprocesses the input image before performing inference.

        Returns:
            image_data: Preprocessed image data ready for inference.
        """
        # Read the input image using OpenCV
        self.img = cv2.imread(self.input_image)

        # Get the height and width of the input image
        self.img_height, self.img_width = self.img.shape[:2]

        # Convert the image color space from BGR to RGB
        img = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)

        # Resize the image to match the input shape
        img = cv2.resize(img, (self.input_width, self.input_height))

        # Normalize the image data by dividing it by 255.0
        image_data = np.array(img) / 255.0

        # Transpose the image to have the channel dimension as the first dimension
        # image_data = np.transpose(image_data, (2, 0, 1))  # Channel first

        # Expand the dimensions of the image data to match the expected input shape
        image_data = np.expand_dims(image_data, axis=0).astype(np.float32)

        # Return the preprocessed image data
        return image_data

    
    def postprocess(self, input_image, output):
        """
        Performs post-processing on the model's output to extract bounding boxes, scores, and class IDs.
        Args:
            input_image (numpy.ndarray): The input image.
            output (numpy.ndarray): The output of the model.

        Returns:
            numpy.ndarray: The input image with detections drawn on it.
        """

        # Transpose and squeeze the output to match the expected shape
        outputs = np.transpose(np.squeeze(output[0]))

        # Get the number of rows in the outputs array
        rows = outputs.shape[0]

        # Lists to store the bounding boxes, scores, and class IDs of the detections
        boxes = []
        scores = []
        class_ids = []
    
        # Calculate the scaling factors for the bounding box coordinates
        x_factor = self.img_width / self.input_width
        y_factor = self.img_height / self.input_height

        # Iterate over each row in the outputs array
        for i in range(rows):
            # Extract the class scores from the current row
            classes_scores = outputs[i][4:]

            # Find the maximum score among the class scores
            max_score = np.amax(classes_scores)

            # If the maximum score is above the confidence threshold
            if max_score >= self.confidence_thres:
                # Get the class ID with the highest score
                class_id = np.argmax(classes_scores)

                # Extract the bounding box coordinates from the current row
                x, y, w, h = outputs[i][0], outputs[i][1], outputs[i][2], outputs[i][3]

                # Calculate the scaled coordinates of the bounding box
                left = int((x - w / 2) * x_factor)
                top = int((y - h / 2) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)

                # Add the class ID, score, and box coordinates to the respective lists
                # and filter indices before NMS
                if class_id in self.positive_list:
                    class_ids.append(class_id)
                    scores.append(max_score)
                    boxes.append([left, top, width, height])

        # Apply non-maximum suppression to filter out overlapping bounding boxes
        indices = cv2.dnn.NMSBoxes(boxes, scores, self.confidence_thres, self.iou_thres)

        self.predresult = []
        # Iterate over the selected indices after non-maximum suppression
        # for i in indices:
        #     # Get the box, score, and class ID corresponding to the index
        #     box = boxes[i]
        #     score = scores[i]
        #     class_id = class_ids[i]
        #     self.predresult.append((class_id,box,score))
        #     # Draw the detection on the input image
        #     self.draw_detections(input_image, box, score, class_id)

        for i in range(len(boxes)):
            # Get the box, score, and class ID corresponding to the index
            box = boxes[i]
            score = scores[i]
            class_id = class_ids[i]
            self.predresult.append((class_id,box,score))
            # Draw the detection on the input image
            self.draw_detections(input_image, box, score, class_id)

        # Return the modified input image
        return input_image

    def pred_result(self):
        return self.predresult
    
    def main(self):
        """
        Performs inference using an ONNX model and returns the output image with drawn detections.

        Returns:
            output_img: The output image with drawn detections.
        """
        # Create an inference session using the ONNX model and specify execution providers
        # session = ort.InferenceSession(self.onnx_model, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
        interpreter = tf.lite.Interpreter(model_path=self.tf_model)
        interpreter.allocate_tensors()
        # Get the model inputs
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        # print(output_details)

        # Store the shape of the input for later use
        input_shape = input_details[0]['shape']
        self.input_width = input_shape[1]
        self.input_height = input_shape[2]

        # Preprocess the image data
        img_data = self.preprocess()

        # Run inference using the preprocessed image data
        interpreter.set_tensor(input_details[0]['index'], img_data )
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        # print(output_data)
        # Perform post-processing on the outputs to obtain output image.
        return self.postprocess(self.img,output_data )  # output image

